In [1]:
import copy
import math
import random
import numpy as np

In [2]:
datas = np.load('data/data.npy')
targets = np.load('data/target.npy')
datas.shape, targets.shape

((10, 20, 20), (10, 20, 20))

In [ ]:
from collections import deque
memory = deque(maxlen=1000)

def append_sample(state, mask, target, reward):
    memory.append((state, mask, target, reward))